In [1]:
import numpy as np
import pandas as pd

In [88]:
csv_file = "matches_2020-07-17T06:55:48.483660+00:00.csv"
csv_data_ = pd.read_csv(csv_file)

In [3]:
csv_data_.head()

,timestamp,ais_distance,ais_bearing,ais_width,ais_length,ai_bearing,imageIndex,y_max_rel,height,height_rel,width,width_rel,vessel_course,mode,match
0,2020-06-28T17:33:00+00:00,2354.8,263.0,16.0,128.0,268.12,2,55.6,19.0,9.5,19.0,1.1,279.2,open_sea,1
1,2020-06-28T18:06:00+00:00,984.9,205.7,16.0,128.0,215.27,1,61.5,29.0,14.4,95.0,5.3,257.1,open_sea,1
2,2020-06-25T06:09:00+00:00,5293.3,82.4,16.0,140.0,87.18,2,52.6,8.0,4.0,14.0,0.8,99.2,open_sea,1
3,2020-06-25T01:46:00+00:00,3555.2,75.9,28.0,185.0,77.56,3,54.1,12.0,6.0,10.0,0.5,71.3,open_sea,1
4,2020-06-25T01:46:00+00:00,19032.5,33.4,24.0,180.0,42.90,2,50.6,8.0,4.0,11.0,0.6,71.3,open_sea,1


In [111]:
csv_data = csv_data_.copy(deep=True)
csv_data["bearing_diff"] = (csv_data["ai_bearing"] - csv_data["ais_bearing"])
csv_data["height_rel"] = (csv_data_["height_rel"] - np.mean(csv_data_["height_rel"]))/np.std(csv_data_["height_rel"])
csv_data["width_rel"] = (csv_data_["width_rel"] - np.mean(csv_data_["width_rel"]))/np.std(csv_data_["width_rel"])
csv_data["y_max_rel"] = (csv_data_["y_max_rel"] - np.mean(csv_data_["y_max_rel"]))/np.std(csv_data_["y_max_rel"])
csv_data["ais_distance"] /= 25000.
csv_data["image_1"] = (csv_data["imageIndex"] == 1).astype(int)
csv_data["image_2"] = (csv_data["imageIndex"] == 2).astype(int)
csv_data["image_3"] = (csv_data["imageIndex"] == 3).astype(int)
csv_data["image_4"] = (csv_data["imageIndex"] == 4).astype(int)
csv_data["image_5"] = (csv_data["imageIndex"] == 5).astype(int)

def normalize_bearing(r):
    bearing = r["ais_bearing"]
    return (bearing - 180)

csv_data["ais_bearing"] = csv_data.apply(normalize_bearing, axis=1)

X = csv_data.drop(columns=["timestamp", "mode", "imageIndex", "match"])#"ais_bearing", "bearing_diff", 'ais_distance', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'vessel_course', 'ais_width', 'ais_length', 'y_max_rel', 'height_rel', 'width_rel'])
y = csv_data["match"]

X.head()

,ais_distance,ais_bearing,ais_width,ais_length,ai_bearing,y_max_rel,height,height_rel,width,width_rel,vessel_course,bearing_diff,image_1,image_2,image_3,image_4,image_5
0,0.094192,83.0,16.0,128.0,268.12,0.233882,19.0,1.260154,19.0,-0.123862,279.2,5.12,0,1,0,0,0
1,0.039396,25.7,16.0,128.0,215.27,1.066413,29.0,2.450877,95.0,3.515504,257.1,9.57,1,0,0,0,0
2,0.211732,-97.6,16.0,140.0,87.18,-0.189439,8.0,-0.076372,14.0,-0.383817,99.2,4.78,0,1,0,0,0
3,0.142208,-104.1,28.0,185.0,77.56,0.022221,12.0,0.409638,10.0,-0.643771,71.3,1.66,0,0,1,0,0
4,0.761300,-146.6,24.0,180.0,42.90,-0.471653,8.0,-0.076372,11.0,-0.557120,71.3,9.50,0,1,0,0,0


Index(['y_max_rel', 'height_rel', 'width_rel', 'match'], dtype='object')


In [119]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# model = RandomForestClassifier()
model = SVC()
X, y = shuffle(X, y)

pipe = Pipeline((('scale', StandardScaler()), ('model', LinearSVC())))


scores = cross_val_score(pipe, X, y, cv=5)
print(scores)
print(scores.mean())


# model.fit(X_train, y_train)
# y_predict = model.predict(X_test)

# X_test.head()

# accuracy_score(y_test.values, y_predict)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[0.8275     0.80451128 0.82957393 0.839599   0.8245614 ]
0.8251491228070176


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
